In [1]:
!pip install pandas openpyxl transformers torch
!pip install llama-cpp-python


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:          

In [2]:
import random

def translate_query(query):
    #prompt = f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return all occurrences of'.\n\n{query}\n\n### Response:"
    #Changes up prompts each time to ensure variety in English conversions
    PROMPTS = [
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return all occurrences of'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Find where'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return issuers that'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return'.\n\n{query}\n\n### Response:"
    ]
    #Error handling 
    try:
        #Chooses a random prompt from the array, queries a response from Mistral, returns response
        prompt = random.choice(PROMPTS).format(query=query)
        response = llm(prompt, max_tokens=575, temperature=0.1)
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(f"Error translating query: '{query}'\nError: {str(e)}")
        return "[TRANSLATION_FAILED]"

In [3]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("esgish_short.xlsx")
queries = df["Esgish"].tolist()
#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries_TEST.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")
    #Avoids overwhelming the system/API
    time.sleep(10) 

llama_perf_context_print:        load time =    5584.72 ms
llama_perf_context_print: prompt eval time =    5584.61 ms /    91 tokens (   61.37 ms per token,    16.29 tokens per second)
llama_perf_context_print:        eval time =    2882.35 ms /    59 runs   (   48.85 ms per token,    20.47 tokens per second)
llama_perf_context_print:       total time =    8475.52 ms /   150 tokens
Llama.generate: 24 prefix-match hit, remaining 31 prompt tokens to eval
llama_perf_context_print:        load time =    5584.72 ms
llama_perf_context_print: prompt eval time =     979.52 ms /    31 tokens (   31.60 ms per token,    31.65 tokens per second)
llama_perf_context_print:        eval time =    1304.58 ms /    27 runs   (   48.32 ms per token,    20.70 tokens per second)
llama_perf_context_print:       total time =    2288.03 ms /    58 tokens
Llama.generate: 24 prefix-match hit, remaining 21 prompt tokens to eval
llama_perf_context_print:        load time =    5584.72 ms
llama_perf_context_print: p

Saved batch 1 to translated_queries_TEST.xlsx
